In [1]:
import pyodbc
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
model = pe.ConcreteModel("Cargacamiones")
import random
random.seed(0)

In [2]:
path=r"C:\Users\juano\Documents\pyomcamiones\Carga de camiones\Transporte_cubicaje.mdb"
conn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+path+';')
cursor = conn.cursor()

Familias Carga de datos

In [3]:
cursor.execute('select IdProducto from Productos')
productos=[]
for row in cursor.fetchall():
    productos.append(row[0])

In [4]:
cursor.execute('select IdFamilia from Familias')
familias=[]
for row in cursor.fetchall():
    familias.append(row[0])

In [5]:
cursor.execute('select IdCamion from Camiones WHERE IdCamion<4')
camiones=[]
for i in range(1,9):
    camiones.append(i)

In [6]:
cursor.execute('select IdDia from Dias WHERE "IdDia"<=19')
dias=[]
for row in cursor.fetchall():
    dias.append(row[0])
        

In [7]:
cursor.execute('select IdDia from Dias')
dias1=[]
for row in cursor.fetchall():
    dias1.append(row[0])



In [8]:
columnas=[1,2]
A1=[6,7]
B1=[4,2]

## Familias Pyomo

In [9]:
model.i = pe.Set(initialize=productos)
model.f = pe.Set(initialize=familias)
model.c = pe.Set(initialize=camiones)
model.t = pe.Set(initialize=dias)
model.t1 = pe.Set(initialize=dias1)
model.col=pe.Set(initialize=columnas)
model.A1=pe.Set(initialize=A1)
model.B1=pe.Set(initialize=B1)

## Carga de datos productos

In [10]:
cursor.execute('select IdFamilia, LotePedido from Familias')
lotepedido={}
for row in cursor.fetchall():
    lotepedido.update({row[0]:row[1]})


In [11]:
cursor.execute('select IdFamilia, LotePedido from Familias')
CosteStock={}
for row in cursor.fetchall():
    CosteStock.update({row[0]:random.randint(1, 10)*0.005})
CosteStock

{1: 0.035,
 2: 0.035,
 3: 0.005,
 4: 0.025,
 5: 0.045,
 6: 0.04,
 7: 0.035,
 8: 0.025,
 9: 0.04,
 10: 0.03,
 11: 0.05,
 12: 0.02,
 13: 0.045,
 14: 0.015,
 15: 0.025}

In [12]:
cursor.execute('select IdProducto , PiezasCont from Productos')
piezascont={}
for row in cursor.fetchall():
    piezascont.update({row[0]:row[1]})

In [13]:
cursor.execute('select IdProducto , LargoContedor from Productos')
piezas_largo_box={}
for row in cursor.fetchall():
    piezas_largo_box.update({row[0]:row[1]})

In [14]:
cursor.execute('select IdProducto , IdFamilia, beta from beta')
b={}
for row in cursor.fetchall():
    b.update({(row[0],row[1]):row[2]})


In [15]:
cursor.execute('select  IdProducto , IdDia, Demanda from Demanda')
demanda={}
for row in cursor.fetchall():
    demanda.update({(row[0],row[1]):row[2]})

In [16]:
# cursor.execute('select IdProducto , StockInicial from Productos')
stockInicial={}
# for row in cursor.fetchall():
#     stockInicial.update({row[0]:row[1]})
cursor.execute('select  IdProducto , Demanda from Demanda where IdDia=1')

for row in cursor.fetchall():
    stockInicial.update({row[0]:row[1]*3})


In [17]:
cursor.execute('select  IdProducto , Valor from Productos')
valor={}
for row in cursor.fetchall():
    valor.update({row[0]:row[1]})

In [18]:
PC={}
for camion in camiones:
    PC.update({camion:1000})

In [19]:
L=78
nu=26/30
NT_lw=5
NT_up=10

In [20]:
model.LotePedido = pe.Param(model.f , initialize=lotepedido)
model.PiezasCont= pe.Param(model.i ,initialize=piezascont)
model.b= pe.Param(model.i, model.f ,initialize=b, default=0)
model.StockInicial= pe.Param(model.i, initialize=stockInicial)
model.demanda= pe.Param(model.i, model.t1,  initialize=demanda)
model.valor=pe.Param(model.i, initialize=stockInicial)
model.PC=pe.Param(model.c, initialize=PC)
model.L=pe.Param(initialize=L)
model.nu=pe.Param(initialize=nu)
model.NT_lw=pe.Param(initialize=NT_lw)
model.NT_up=pe.Param(initialize=NT_up)
model.LargoCont=pe.Param(model.i,initialize=piezas_largo_box)
model.CS=pe.Param(model.i, initialize=CosteStock)

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (LargoCont) to be 'Any', you can suppress
    this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0) (called from
    C:\Users\juano\Anaconda3\envs\pyomo-tutorials\lib\site-
    packages\pyomo\core\base\param.py:77)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (LargoCont) to be 'Any', you can suppress
    this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0) (called from
    C:\Users\juano\Anaconda3\envs\pyomo-tutorials\lib\site-
    packages\pyomo\core\base\param.py:77)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (LargoCont) to be 'Any', you can suppress
    this warning by explicitly specify

# # Varibles

In [21]:
model.CantidadPedir = pe.Var(model.i, model.c, model.t,model.col, domain=pe.Integers)

In [22]:
model.CantidadPedir2 = pe.Var(model.i,model.f, model.c, model.t,model.col, domain=pe.Integers, bounds=(0,None) )
model.ContenedoresCamion=pe.Var(model.c,model.t,model.col,model.i, domain=pe.Integers)

In [23]:
model.Stock=pe.Var(model.i,model.t, bounds=(0,None))

In [24]:
model.RetrasoDemanda=pe.Var(model.i,model.t, bounds=(0,None))

In [25]:
model.K=pe.Var(model.f,model.c,model.t,model.col ,domain=pe.Integers)

In [26]:
model.Y=pe.Var(model.c,model.t ,domain=pe.Binary, initialize=0)

In [27]:
# model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(5.5,7.5))
# model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(1.5,4.5))
model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(0,None))
model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(0,None))


## Función Objetivo
Minimizar el número de camiones usado
$$\sum_{c,t}PC(c)Y(c,t)+ \sum CS(i)Stock(i,t)$$

In [28]:
expr = sum(model.PC[c]*model.Y[c,t] +sum(model.CS[i]*model.Stock[i,t] for i in model.i) for c in model.c for t in model.t)

In [29]:
 #Coste Transporte
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

## Funciones auxiliares

In [30]:
def UsoCamion(model):
    return sum(model.L*model.Y[c,t] for c in model.c for t in model.t)-\
        sum(model.ContenedoresCamion[c,t] for c in model.c for t in model.t)

In [31]:
def CTransporte(model):
    return sum(model.PC[c]*model.Y[c,t] for c in model.c for t in model.t)

In [32]:
def NumeroCamiones(model):
    return sum(model.Y[c,t] for c in model.c for t in model.t)

## Restricciones

In [33]:
model.stocks=pe.ConstraintList()

$$Stock(i,t)=Stock(i,t-1)+\sum_{c,col}CantidadPedir(i,c,t,col)-Demanda(i,t)$$

In [34]:
for i in model.i:
        lhs=model.Stock[i,1]
        rhs=model.StockInicial[i]-model.demanda[i,1]+sum(model.CantidadPedir[i,c,1,col] for c in model.c for col in model.col )
        model.stocks.add(lhs==rhs)

In [35]:
for t in list(model.t)[1:]:
    for i in model.i:
            lhs=model.Stock[i,t]
            rhs=model.Stock[i,t-1]-model.demanda[i,t-1]+sum(model.CantidadPedir[i,c,t-1,col] for c in model.c for col in model.col)
            model.stocks.add(lhs==rhs)

In [36]:
model.RcantidadPedir=pe.ConstraintList()

$$CantidadPedir2_{i,f,c,t,col}=K(f,c,t,col)*LotePedido(f)*b(i,f)$$

In [37]:
for i in model.i:
    for f in model.f:
        for c in model.c:
            for t in model.t:
                for col in model.col:
                    lhs=model.CantidadPedir2[i,f,c,t,col]
                    rhs=model.K[f,c,t,col]*model.LotePedido[f]*model.b[i,f]
                    model.RcantidadPedir.add(lhs==rhs)

In [38]:
model.RcantidadPedir2=pe.ConstraintList()

$$CantidadPedir_{i,c,t,col}= \sum_f CanitdadPedir2(i,f,c,t,col)$$

In [39]:
for i in model.i:
    for c in model.c:
        for t in model.t:
            for col in model.col:
                lhs=model.CantidadPedir[i,c,t,col]
                rhs=sum(model.CantidadPedir2[i,f,c,t,col] for f in model.f)
                model.RcantidadPedir2.add(lhs==rhs)

In [40]:
# model.RContendores=pe.ConstraintList()

In [41]:
# for c in model.c:
#     for t in model.t:
#         for col in model.col:
#             for i in model.i:
#                 lhs=model.ContenedoresCamion[c,t,col,i]
#                 rhs=model.CantidadPedir[i,c,t,col]/model.PiezasCont[i]
#                 model.RContendores.add(lhs==rhs)

In [42]:
model.RDimensionesCamion=pe.ConstraintList()

$$ 1600A_{c,col,t} +1000B_{c,col,t}   \leq 13000$$

In [43]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=13600
            rhs=(model.A[c,col,t]*1600+model.B[c,col,t]*1000)*model.Y[c,t]
            model.RDimensionesCamion.add(lhs>=rhs)

In [44]:
model.RCajasTotales=pe.ConstraintList()

$$ \sum_i CantidadPedir(i,c,t,col)/PiezasCont(i) \leq A_{c,col,t}*4+B_{c,col,t}*3 $$

In [45]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]/model.PiezasCont[i] for i in model.i)
            rhs=(model.A[c,col,t]*4+model.B[c,col,t]*3)*model.Y[c,t]
            model.RCajasTotales.add(lhs<=rhs)

In [46]:
model.RCajasTipoA=pe.ConstraintList()

$$ \sum_i CantidadPedir(i,c,t,col)/PiezasCont(i)[tipoA] \leq A_{c,col,t}*4 $$

In [47]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]/model.PiezasCont[i]*float(int(model.LargoCont[i])==int(1600)) for i in model.i)
            rhs=model.A[c,col,t]*4*model.Y[c,t]
            model.RCajasTipoA.add(lhs<=rhs)

In [48]:
model.RCajasTipoB=pe.ConstraintList()

In [49]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]/model.PiezasCont[i]*float(int(model.LargoCont[i])==int(1000)) for i in model.i)
            rhs=model.B[c,col,t]*3*model.Y[c,t]
            model.RCajasTipoB.add(lhs<=rhs)

In [50]:
# model.RDimensionesCamion=pe.ConstraintList()

In [51]:
# for c in model.c:
#     for t in model.t:
#         lhs=model.ContenedoresCamion[c,t]
#         rhs=model.L*model.Y[c,t]
#         model.RDimensionesCamion.add(lhs<=rhs)

In [52]:
# model.RLlenado=pe.ConstraintList()
# for c in model.c:
#     for t in model.t:
#         lhs=model.ContenedoresCamion[c,t]
#         rhs=model.L*model.nu*model.Y[c,t]
#         model.RLlenado.add(lhs>=rhs)

In [53]:
model.RCobertura=pe.ConstraintList()
for i in model.i:
    for t in range(1,16):
        lhs=model.Stock[i,t]
        rhs=(model.demanda[i,t+1]+model.demanda[i,t+2]+model.demanda[i,t+3]+model.demanda[i,t+4])
        model.RCobertura.add(lhs>=rhs)
for i in model.i:
    for t in range(16,20):
        lhs=model.Stock[i,t]
        model.RCobertura.add(lhs>=rhs)

In [54]:
model.RCVolumenminimo=pe.ConstraintList()

In [55]:
for t in model.t:
    for c in model.c:
        lhs=(sum(model.CantidadPedir[i,c,t,col]/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col]/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/(3000*2400*13600)
        rhs=0.9*model.Y[c,t]
        model.RCVolumenminimo.add(lhs >= rhs)

In [56]:
model.bounds=pe.ConstraintList()
for i in model.i:
    for t in model.t:
        model.bounds.add(model.Stock[i,t]>=0)


In [57]:
solver = po.SolverFactory('gurobi')
solver.options["MIPFocus"] = 0
solver.options["MIPGap"] = 0.00125
results = solver.solve(model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-06
Read LP format model from file C:\Users\juano\AppData\Local\Temp\tmp55kximnu.pyomo.lp
Reading time = 0.22 seconds
x83411: 73968 rows, 79478 columns, 156318 nonzeros
Set parameter MIPGap to value 0.00125
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 73968 rows, 79478 columns and 156318 nonzeros
Model fingerprint: 0x6f1d8380
Model has 1216 quadratic constraints
Variable types: 286 continuous, 79192 integer (152 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+00]
  QMatrix range    [3e+00, 2e+03]
  QLMatrix range   [2e-02, 1e+00]
  Objective range  [4e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
  QRHS range       [1e+04, 1e+04]
Presolve removed 73539 rows and 73289 columns
Presolve time: 0.05s
Presolved: 2733 rows, 7917 columns, 22827 nonzeros

In [58]:
def mean(x):
    return sum(x)/len(x)

Best objective 1.772093200000e+05, best bound 1.754607360233e+05, gap 0.9867%
0.9507181523246101
69

In [59]:
utilizacion=0
volumen_por_camion=[]
for t in model.t:
    for c in model.c:
        if model.Y[c,t].value==1:
            volumen_por_camion.append(((sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600)))
volumen_por_camion
print(mean(volumen_por_camion))
print(len(volumen_por_camion))

0.9664851095295508
68


In [60]:
for t in model.t:
    for c in model.c:

            if model.Y[c,t].value==1:
                print(t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
            +(sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600)))

1 6.0 4.0 0.90017082590612
1 8.0 0.0 0.9004010695187165
1 8.0 0.0 0.9001736111111112
1 6.0 4.0 0.9357843137254903
1 8.0 0.0 0.9606729055258467
1 8.0 0.0 0.9339795008912656
1 5.0 5.0 0.9002181743909685
1 6.0 4.0 0.935650623885918
3 6.0 4.0 0.9998143196672608
3 6.0 4.0 0.9849116161616162
4 6.0 4.0 0.9985377673796791
4 8.0 0.0 0.9696078431372548
4 8.0 0.0 0.9704257650029707
5 6.0 4.0 0.966726084373143
5 6.0 4.0 0.9995682932263814
6 6.0 4.0 1.0
6 6.0 4.0 0.999814319667261
7 8.0 0.0 0.9696524064171124
7 6.0 4.0 0.9975573752228163
7 7.0 2.0 0.9534313725490197
8 6.0 4.0 0.9997957516339869
8 6.0 4.0 0.9997549019607843
9 6.0 4.0 0.9978386809269162
9 7.0 2.0 0.9824346405228758
10 7.0 2.0 0.9818404634581105
10 6.0 4.0 0.9828078579916816
10 8.0 0.0 0.9557040998217469
11 6.0 4.0 0.9852941176470589
11 6.0 4.0 0.9997493315508021
12 8.0 0.0 0.9502673796791444
12 8.0 0.0 0.9411764705882353
12 8.0 0.0 0.9411764705882355
12 8.0 0.0 0.9409313725490196
12 8.0 0.0 0.9697712418300654
12 6.0 4.0 0.99985795454

In [61]:
0.8628883539414994
79

79

In [62]:
int(model.LargoCont[14])

1000

In [63]:
for i,v in model.LargoCont:
    print(i)

TypeError: cannot unpack non-iterable int object